In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras_nlp

2024-11-21 21:29:12.289509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732199352.339091   69846 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732199352.354445   69846 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 21:29:12.463921: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ryanfikri/anaconda3/envs/capstone-project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not foun

In [10]:
# Initialize the subword tokenizer
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary='./mental_vocab_subwords.txt'
)

# initialize model
model = tf.keras.models.load_model('first_iteration.keras')

# initialize label encoder
# Load the vocabulary from a file
with open("label_vocabulary.txt", "r") as f:
    vocab = [line.strip() for line in f]

def padding_func(sequences):
  '''Generates padded sequences from a tf.data.Dataset'''

  # Put all elements in a single ragged batch
  sequences = sequences.ragged_batch(batch_size=sequences.cardinality())

  # Output a tensor from the single batch
  sequences = sequences.get_single_element()

  # Pad the sequences
  padded_sequences = tf.keras.utils.pad_sequences(sequences.numpy(), 
                                                  maxlen=500, 
                                                  truncating='post', 
                                                  padding='post'
                                                 )

  # Convert back to a tf.data.Dataset
  padded_sequences = tf.data.Dataset.from_tensor_slices(padded_sequences)

  return padded_sequences

In [11]:
rawtext_test = ["i want to die"]
sequence_test = padding_func(tf.data.Dataset.from_tensors(tokenizer.tokenize(rawtext_test)))
for element in sequence_test:
    print(element)
predictions = model.predict(sequence_test)

# predictions will be a numpy array of shape (1, num_classes) with probabilities for each class
print(predictions)

# To get the predicted class index
predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]
print(f"Predicted class: {vocab[predicted_class_index]}")

tf.Tensor(
[[ 10  83  21 488 238]
 [  0   0   0   0   0]
 [  0   0   0   0   0]
 ...
 [  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0   0   0   0]], shape=(500, 5), dtype=int32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
[[9.7543848e-01 3.3598391e-11 7.2672541e-05 ... 2.4488727e-02
  8.1130267e-08 3.5747860e-10]
 [2.7726456e-08 4.6485100e-02 6.1314758e-02 ... 2.3749687e-12
  3.1681042e-04 1.4108206e-04]
 [2.7726456e-08 4.6485100e-02 6.1314758e-02 ... 2.3749687e-12
  3.1681042e-04 1.4108206e-04]
 ...
 [2.7726456e-08 4.6485100e-02 6.1314758e-02 ... 2.3749687e-12
  3.1681042e-04 1.4108206e-04]
 [2.7726456e-08 4.6485100e-02 6.1314758e-02 ... 2.3749687e-12
  3.1681042e-04 1.4108206e-04]
 [2.7726456e-08 4.6485100e-02 6.1314758e-02 ... 2.3749687e-12
  3.1681042e-04 1.4108206e-04]]
Predicted class: Suicidal
